# Modeling
### [1] 원핫 없이 통으로 학습 & 종목명 원핫 적용 후 학습 비교
### [2] 머신러닝 => 전체 돌리는 모듈로 상위 3개 정도 & 딥러닝 모델 비교

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [ ]:
# stock_df = pd.DataFrame()

# files_m = os.listdir('./temp_data/A000020.csv')

# for min_df in files_m[:1]:
#     _df = pd.read_csv(f'./temp_data/A000020.csv/{min_df}')
#     stock_df = pd.concat([stock_df, _df])
#     print(min_df)

In [2]:
stock_df = pd.read_csv('../data_min/A000020.csv')

In [3]:
# 첫 5줄을 봅니다.
stock_df

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
0,20221121,1530,8920.0,8920.0,8920.0,8920.0,4128.0,36820000.0,21105.0,101675.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
1,20221121,1520,8920.0,8920.0,8920.0,8920.0,605.0,5390000.0,16977.0,101675.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
2,20221121,1519,8920.0,8920.0,8920.0,8920.0,323.0,2880000.0,16977.0,101070.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
3,20221121,1518,8920.0,8920.0,8920.0,8920.0,734.0,6550000.0,16977.0,100747.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
4,20221121,1517,8920.0,8920.0,8920.0,8920.0,316.0,2820000.0,16977.0,100013.0,...,249144000000,27931470,26667255,1264215,4.53,20221121,100.0,-8301,83679,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232666,20201028,905,17650.0,17700.0,17600.0,17700.0,629.0,11120000.0,22042.0,3156.0,...,529292000000,27931470,26389845,1541625,5.52,20201028,100.0,12546,128337,9.6
232667,20201028,904,17500.0,17650.0,17500.0,17650.0,6448.0,113390000.0,21598.0,2971.0,...,529292000000,27931470,26389845,1541625,5.52,20201028,100.0,12546,128337,9.9
232668,20201028,903,17600.0,17600.0,17500.0,17500.0,1308.0,22930000.0,15487.0,2634.0,...,529292000000,27931470,26389845,1541625,5.52,20201028,100.0,12546,128337,10.2
232669,20201028,902,17550.0,17650.0,17550.0,17600.0,2429.0,42710000.0,14780.0,2033.0,...,529292000000,27931470,26389845,1541625,5.52,20201028,100.0,12546,128337,9.9


In [8]:
# def data_type_trans(stock_df):

#     stock_df = stock_df.astype({'시간':'uint16'})
#     stock_df = stock_df.astype({'시가':'uint32'})
#     stock_df = stock_df.astype({'고가':'uint32'})
#     stock_df = stock_df.astype({'저가':'uint32'})
#     stock_df = stock_df.astype({'종가':'uint32'})
#     stock_df = stock_df.astype({'전일대비':'int32'})
#     stock_df = stock_df.astype({'거래량':'uint32'})
#     stock_df = stock_df.astype({'거래대금':'uint64'})
#     stock_df = stock_df.astype({'누적체결매도수량':'uint64'})
#     stock_df = stock_df.astype({'상장주식수':'uint64'})
#     stock_df = stock_df.astype({'시가총액':'uint64'})
#     stock_df = stock_df.astype({'외국인주문한도수량':'uint64'})
#     stock_df = stock_df.astype({'외국인주문가능수량':'uint64'})
#     stock_df = stock_df.astype({'외국인현보유수량':'uint64'})
#     stock_df = stock_df.astype({'외국인현보유비율':'float32'})
#     stock_df = stock_df.astype({'수정주가일자':'uint32'})
#     stock_df = stock_df.astype({'수정주가비율':'float32'})
#     stock_df = stock_df.astype({'기관순매수량':'int64'})
#     stock_df = stock_df.astype({'기관누적순매수량':'int64'})
#     stock_df = stock_df.drop(['등락주선', '등락비율', '예탁금', '주식회전율', '거래성립률'], axis=1)
#     stock_df = stock_df.astype({'label':'uint32'})

#     stock_df.info()

#     return stock_df

In [62]:
# trans = stock_df.loc[:,['고가','label']]
# trans = trans.rename(columns={'label':'pct_label'}).T

In [63]:
# get_trans = trans.pct_change().T.iloc[:,-1]
# get_trans

20221103    0.000000
20221103    0.000000
20221103    0.000000
20221103    0.000000
20221103    0.000000
              ...   
20201029    0.003861
20201029    0.005145
20201029    0.006433
20201029    0.007725
20201029    0.009019
Name: pct_label, Length: 189537, dtype: float64

In [64]:
# stock_df = pd.concat([stock_df, get_trans], axis=1)
# stock_df = stock_df.drop(['label'], axis=1)
# stock_df



,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
20221103,1530,216500,216500,216500,216500,0,12129,2625928000,210922,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1520,216800,216900,216700,216800,0,863,187072000,210922,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1519,216900,216900,216700,216800,0,1106,239789000,210524,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1518,216900,216900,216800,216800,0,461,99981000,210091,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221103,1517,216600,216900,216600,216900,0,601,130274000,209993,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201029,905,37509,37557,37365,37461,0,1387,51942000,5773,0,0,0,0,0,0.0,0,0.0,0,0,0.003861
20201029,904,37413,37509,37317,37461,0,1810,67708000,4798,0,0,0,0,0,0.0,0,0.0,0,0,0.005145
20201029,903,37413,37461,37269,37461,0,2672,99978000,4464,0,0,0,0,0,0.0,0,0.0,0,0,0.006433
20201029,902,37221,37413,36980,37413,0,5894,219176000,3571,0,0,0,0,0,0.0,0,0.0,0,0,0.007725


In [65]:
# stock_df['pct_label'] = stock_df['pct_label'].mul(100)
# stock_df['pct_label'] = stock_df['pct_label'].round(0)
# stock_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
20221103,1530,216500,216500,216500,216500,0,12129,2625928000,210922,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1520,216800,216900,216700,216800,0,863,187072000,210922,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1519,216900,216900,216700,216800,0,1106,239789000,210524,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1518,216900,216900,216800,216800,0,461,99981000,210091,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221103,1517,216600,216900,216600,216900,0,601,130274000,209993,0,0,0,0,0,0.0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201029,905,37509,37557,37365,37461,0,1387,51942000,5773,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20201029,904,37413,37509,37317,37461,0,1810,67708000,4798,0,0,0,0,0,0.0,0,0.0,0,0,1.0
20201029,903,37413,37461,37269,37461,0,2672,99978000,4464,0,0,0,0,0,0.0,0,0.0,0,0,1.0
20201029,902,37221,37413,36980,37413,0,5894,219176000,3571,0,0,0,0,0,0.0,0,0.0,0,0,1.0


In [4]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
# stock_df = stock_df.drop(columns=['상장주식수', '시가총액', '외국인주문한도수량', '외국인주문가능수량', '외국인현보유수량', '외국인현보유비율', '수정주가일자', '수정주가비율', '기관순매수량', '기관누적순매수량']
#             , axis=1
# )

X_stock_df = stock_df.iloc[:,0:-1]
y_stock_df = stock_df.iloc[:,-1]


In [5]:
from sklearn.model_selection import train_test_split

X_stock_sc = min_max_scaler.fit_transform(X_stock_df)

X_stock_trans_df = pd.DataFrame(data=X_stock_sc, columns=X_stock_df.columns)
X_stock_trans_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232671 entries, 0 to 232670
Data columns (total 21 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   날짜         232671 non-null  float64
 1   시간         232671 non-null  float64
 2   시가         232671 non-null  float64
 3   고가         232671 non-null  float64
 4   저가         232671 non-null  float64
 5   종가         232671 non-null  float64
 6   거래량        232671 non-null  float64
 7   거래대금       232671 non-null  float64
 8   누적체결매수수량   232671 non-null  float64
 9   누적체결매도수량   232671 non-null  float64
 10  전일대비       232671 non-null  float64
 11  상장주식수      232671 non-null  float64
 12  시가총액       232671 non-null  float64
 13  외국인주문한도수량  232671 non-null  float64
 14  외국인주문가능수량  232671 non-null  float64
 15  외국인현보유수량   232671 non-null  float64
 16  외국인현보유비율   232671 non-null  float64
 17  수정주가일자     232671 non-null  float64
 18  수정주가비율     232671 non-null  float64
 19  기관순매수량     232671 non-n

In [6]:
y_stock_df.value_counts()

0.0     51922
0.3     35282
0.4     19322
0.7     16930
1.0     10929
        ...  
18.5        1
18.4        1
15.6        1
14.6        1
20.7        1
Name: pct_label, Length: 200, dtype: int64

In [7]:
y_stock_df = to_categorical(y_stock_df)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_stock_sc, y_stock_df, test_size=0.2, shuffle=True, random_state=42)

In [9]:
# X_stock_trans_df 데이터 변환 해줘야 함
X_stock_trans_df

,날짜,시간,시가,고가,저가,종가,거래량,거래대금,누적체결매수수량,누적체결매도수량,...,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량
0,1.0,0.862826,0.049912,0.048766,0.051069,0.050322,0.003583,0.001330,0.065388,0.010165,...,0.0,0.047679,0.0,0.449141,0.550859,0.550971,1.0,0.0,0.405219,0.632240
1,1.0,0.849108,0.049912,0.048766,0.051069,0.050322,0.000525,0.000195,0.064903,0.010165,...,0.0,0.047679,0.0,0.449141,0.550859,0.550971,1.0,0.0,0.405219,0.632240
2,1.0,0.847737,0.049912,0.048766,0.051069,0.050322,0.000280,0.000104,0.064903,0.010104,...,0.0,0.047679,0.0,0.449141,0.550859,0.550971,1.0,0.0,0.405219,0.632240
3,1.0,0.846365,0.049912,0.048766,0.051069,0.050322,0.000637,0.000237,0.064903,0.010072,...,0.0,0.047679,0.0,0.449141,0.550859,0.550971,1.0,0.0,0.405219,0.632240
4,1.0,0.844993,0.049912,0.048766,0.051069,0.050322,0.000274,0.000102,0.064903,0.009999,...,0.0,0.047679,0.0,0.449141,0.550859,0.550971,1.0,0.0,0.405219,0.632240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232666,0.0,0.005487,0.562537,0.552496,0.566508,0.564073,0.000546,0.000402,0.065498,0.000316,...,0.0,0.685315,0.0,0.208030,0.791970,0.791262,0.0,0.0,0.574178,0.766744
232667,0.0,0.004115,0.553729,0.549627,0.560570,0.561147,0.005597,0.004096,0.065446,0.000297,...,0.0,0.685315,0.0,0.208030,0.791970,0.791262,0.0,0.0,0.574178,0.766744
232668,0.0,0.002743,0.559601,0.546758,0.560570,0.552370,0.001135,0.000828,0.064728,0.000263,...,0.0,0.685315,0.0,0.208030,0.791970,0.791262,0.0,0.0,0.574178,0.766744
232669,0.0,0.001372,0.556665,0.549627,0.563539,0.558221,0.002109,0.001543,0.064645,0.000203,...,0.0,0.685315,0.0,0.208030,0.791970,0.791262,0.0,0.0,0.574178,0.766744


In [10]:
X_train

array([[0.45274474, 0.7654321 , 0.52143277, ..., 0.        , 0.45959395,
        0.58748091],
       [0.96974071, 0.01783265, 0.20140928, ..., 0.        , 0.48797666,
        0.33581912],
       [0.98999652, 0.07956104, 0.05695831, ..., 0.        , 0.47265875,
        0.50738658],
       ...,
       [0.49196237, 0.45816187, 0.40399295, ..., 0.        , 0.49918548,
        0.33745155],
       [0.48270542, 0.29903978, 0.374633  , ..., 0.        , 0.48101471,
        0.53463486],
       [0.49723784, 0.72839506, 0.54492073, ..., 0.        , 0.47248855,
        0.39533042]])

In [11]:
# 레이블 카테고리 개수
y_stock_df.shape

(232671, 30)

In [12]:
# y_stock_df.value_counts()

## 3. 원-핫 인코딩

In [75]:
# # 원-핫 인코딩 처리를 합니다.
# y_train = pd.get_dummies(y_train)
# y_test = pd.get_dummies(y_test)

## 4. 소프트맥스

In [76]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, LSTM

# # 모델 설정
# model = Sequential()
# model.add(Dense(1000,  input_dim=X_stock_df.shape[1], activation='relu'))
# model.add(LSTM(X_stock_df.shape[1], activation='tanh'))
# model.add(Dense(len(np.max(y_train)), activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
# model.summary()

# # 모델 컴파일
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# # 모델 실행
# history=model.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_test, y_test))

# model.evaluate(X_test, y_test)[1]

In [77]:
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

In [13]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [14]:
X_train.shape

(186136, 21)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

X_train_lstm = X_train.reshape(X_train.shape[0], 21, 1)
X_test_lstm = X_test.reshape(X_test.shape[0], 21, 1)


In [16]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
print(y_train.shape)
print(X_train_lstm.shape)
print(X_train_lstm)

(186136, 30)
(186136, 21, 1)
[[[0.45274474]
  [0.7654321 ]
  [0.52143277]
  ...
  [0.        ]
  [0.45959395]
  [0.58748091]]

 [[0.96974071]
  [0.01783265]
  [0.20140928]
  ...
  [0.        ]
  [0.48797666]
  [0.33581912]]

 [[0.98999652]
  [0.07956104]
  [0.05695831]
  ...
  [0.        ]
  [0.47265875]
  [0.50738658]]

 ...

 [[0.49196237]
  [0.45816187]
  [0.40399295]
  ...
  [0.        ]
  [0.49918548]
  [0.33745155]]

 [[0.48270542]
  [0.29903978]
  [0.374633  ]
  ...
  [0.        ]
  [0.48101471]
  [0.53463486]]

 [[0.49723784]
  [0.72839506]
  [0.54492073]
  ...
  [0.        ]
  [0.47248855]
  [0.39533042]]]


In [18]:
y_train.shape

(186136, 30)

In [19]:
y_test.shape

(46535, 30)

In [20]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [21]:

# 모델 설정
model = Sequential()
model.add(LSTM(100, input_shape=(21, 1), activation='tanh'))
model.add(Dense(30, activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X_train_lstm, y_train, epochs=10, batch_size=200,verbose=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 30)                3030      
                                                                 
Total params: 43,830
Trainable params: 43,830
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
931/931 [==============================] - 27s 28ms/step - loss: 1.0690 - accuracy: 0.6986
Epoch 2/10
931/931 [==============================] - 26s 28ms/step - loss: 1.0288 - accuracy: 0.6992
Epoch 3/10
931/931 [==============================] - 26s 28ms/step - loss: 1.0195 - accuracy: 0.6992
Epoch 4/10
931/931 [==============================] - 26s 28ms/step - loss: 0.9797 - accuracy: 0.6993
Epoch 5/10
931/931 [====================

In [47]:
model.evaluate(X_test, y_test)[1]

0.7138068079948425

In [48]:
predict = model.predict(X_test)

1455/1455 [==============================] - 5s 3ms/step


In [49]:
predict[0]

array([6.3224030e-01, 2.6978216e-01, 6.0484692e-02, 2.6280599e-02,
       5.7714609e-03, 2.4538008e-03, 1.3107358e-03, 6.4932340e-04,
       4.7613916e-04, 8.6917702e-05, 1.0630668e-04, 5.9439029e-05,
       2.8661638e-04, 3.1551381e-06, 1.1152962e-06, 5.4803769e-07,
       1.0371655e-07, 2.6900693e-07, 9.2528481e-08, 1.8171394e-07,
       1.5282910e-06, 9.5656173e-07, 6.7619561e-07, 6.5209758e-07,
       5.3779348e-07, 3.9681004e-07, 9.8080150e-08, 8.2230827e-07,
       4.4638597e-07, 2.3792190e-08], dtype=float32)

In [50]:
predict.index(predict[0][0].max())

AttributeError: 'numpy.ndarray' object has no attribute 'index'

In [51]:
# for i,rate  in enumerate(predict):
#     for i, ra in enumerate(rate):
#         if rate.max() == ra:
#             print(i)

print(list(map(np.argmax, predict)))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 26, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,